In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"dipaktripathi","key":"52f9b4af631a74adb6c90f57134627f9"}'}

In [ ]:
 ! mkdir ~/.kaggle
 ! cp kaggle.json ~/.kaggle/
 ! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!unzip '/content/imdb-dataset-of-50k-movie-reviews.zip'

imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: IMDB Dataset.csv        


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.shape

(50000, 2)

Gettings only 20k records to process faster

In [ ]:
df_final = pd.concat([df[df.sentiment=='positive'].head(10000), df[df.sentiment=='negative'].head(10000)])

In [ ]:
df_final.shape

(20000, 2)

In [ ]:
df_final.sentiment.value_counts()

negative    10000
positive    10000
Name: sentiment, dtype: int64

In [ ]:
stopword_list = stopwords.words('english')

In [ ]:
def clean_function(text):
  #text  = text.lower()
  text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  text = re.sub('\[[^]]*\]', '', text)
  ps = nltk.porter.PorterStemmer()
  text = ' '.join([ps.stem(word) for word in text.split()])
  return text

In [ ]:
df_final['review'] = df_final['review'].apply(clean_function)

In [ ]:
def remove_stopwords(text):
  tokens = word_tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token.lower() for token in tokens if token not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

In [ ]:
df_final['review'] = df_final['review'].apply(remove_stopwords)

In [ ]:
df_final.head()

,review,sentiment
0,one review ha mention watch 1 oz episod youll ...,positive
1,a wonder littl product br br film techniqu ver...,positive
2,i thought thi wa wonder way spend time hot sum...,positive
4,petter mattei love time money visual stun film...,positive
5,probabl alltim favorit movi stori selfless sac...,positive


In [ ]:

df_final['review'][0]

'one review ha mention watch 1 oz episod youll hook right thi exactli happen mebr br first thing struck oz wa brutal unflinch scene violenc set right word go trust thi show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordbr br it call oz nicknam given oswald maximum secur state penitentari it focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awaybr br i would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod i ever saw struck nasti wa surreal i couldnt say i wa readi i watch i develop tast oz got accustom high level graphic violenc violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due 

In [ ]:
label_encoder = LabelEncoder() 
df_final['sentiment']= label_encoder.fit_transform(df_final['sentiment']) 

In [ ]:
df_final.sentiment.shape

(20000,)

In [ ]:
df_final.to_csv('final.csv')

In [ ]:
X = df_final['review']
y = df_final['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16000,), (4000,), (16000,), (4000,))

In [ ]:
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))

In [ ]:
tv_x_train = tv.fit_transform(X_train)
tv_x_test = tv.transform(X_test)

In [ ]:
tv_x_train.shape, tv_x_test.shape

((16000, 2783686), (4000, 2783686))

In [ ]:
from sklearn import decomposition, preprocessing
from sklearn.decomposition import TruncatedSVD

In [ ]:
svd = decomposition.TruncatedSVD(n_components=120)

In [ ]:
svd.fit(tv_x_train)
xtrain_svd = svd.transform(tv_x_train)
xvalid_svd = svd.transform(tv_x_test)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [ ]:
classifier = SVC(C=1.0, probability=True)
classifier.fit(xtrain_svd_scl, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
classifier_predict=classifier.predict(xvalid_svd_scl)
print(classifier_predict)

[0 0 0 ... 0 0 0]


In [ ]:
print(accuracy_score(y_test, classifier_predict))

0.50175


In [ ]:
print('Classification Report')
print(classification_report(y_test, classifier_predict ,target_names=['Positive','Negative']))

Classification Report
              precision    recall  f1-score   support

    Positive       0.50      1.00      0.67      2000
    Negative       1.00      0.00      0.01      2000

    accuracy                           0.50      4000
   macro avg       0.75      0.50      0.34      4000
weighted avg       0.75      0.50      0.34      4000



In [ ]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(tv_x_train, y_train)
predictions = clf.predict(tv_x_test)

In [ ]:
predictions

array([1, 0, 1, ..., 1, 1, 0])

In [ ]:
report=classification_report(y_test,predictions,target_names=['Positive','Negative'])
print(report)

              precision    recall  f1-score   support

    Positive       0.72      0.75      0.73      2000
    Negative       0.74      0.71      0.72      2000

    accuracy                           0.73      4000
   macro avg       0.73      0.73      0.73      4000
weighted avg       0.73      0.73      0.73      4000



In [ ]:
print(accuracy_score(y_test, predictions))

0.728
